In [1]:
from tqdm import tqdm
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

/var/data/python/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df_anomaly = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._anomaly.csv',sep=';')
df_anomaly.replace('        -', np.nan, inplace=True)
df_anomaly.replace('-', np.nan, inplace=True)

/tmp/ipykernel_324/1735381011.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_anomaly.replace('        -', np.nan, inplace=True)
/tmp/ipykernel_324/1735381011.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_anomaly.replace('-', np.nan, inplace=True)


In [3]:
df_problems = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._problems.csv',sep=';')
df_problems.replace('        -', np.nan, inplace=True)
df_problems.replace('-', np.nan, inplace=True)

/tmp/ipykernel_324/3875790439.py:1: DtypeWarning: Columns (3,11,12,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_problems = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._problems.csv',sep=';')
/tmp/ipykernel_324/3875790439.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_problems.replace('        -', np.nan, inplace=True)
/tmp/ipykernel_324/3875790439.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
 

In [4]:
df_normal = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._normal.csv',sep=';')
df_normal.replace('        -', np.nan, inplace=True)
df_normal.replace('-', np.nan, inplace=True)

/tmp/ipykernel_324/4066241437.py:1: DtypeWarning: Columns (3,12,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_normal = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._normal.csv',sep=';')
/tmp/ipykernel_324/4066241437.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_normal.replace('        -', np.nan, inplace=True)
/tmp/ipykernel_324/4066241437.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_norma

Метка 0 - нормальное состояние, 1 - аномалия, 2 - поломка

In [5]:
df_normal['target'] = 0
df_anomaly['target'] = 1
df_problems['target'] = 2

In [6]:
train = pd.concat([
    df_normal,
    df_anomaly,
    df_problems
])

In [7]:
column_names_train = train.columns.tolist()
for i in train:
  if (train[i].isnull().sum() / len(train[i])) * 100 > 90:
    del train[i]

In [8]:
train.dropna(thresh=5, inplace=True, axis=0)

In [9]:
for i in train.select_dtypes(include=['object']):
   train[i] = train[i].fillna(train[i].mode().iloc[0])

In [10]:
def make_harmonic_features(value, period=24):
    value *= 2 * np.pi / period
    return np.cos(value), np.sin(value)

In [11]:
train['Темп.масла двиг.,°С'] = train['Темп.масла двиг.,°С'].str.replace(',', '.').astype('float')
train['Обор.двиг.,об/мин'] = train['Обор.двиг.,об/мин'].str.replace(',', '.').astype('float')
train['Полож.пед.акселер.,%'] = train['Полож.пед.акселер.,%'].str.replace(',', '.').astype('float')

In [12]:
train_KPP = train[['КПП. Температура масла','КПП. Давление масла в системе смазки','Нейтраль КПП (spn3843)','Засоренность фильтра КПП (spn3847)','Сост.пед.сцепл.','Давление в пневмостистеме (spn46), кПа','Недопустимый уровень масла в гидробаке (spn3850)','Аварийная температура масла в гидросистеме (spn3849)','Аварийная температура масла ГТР (spn3867)', 'target']]
train_KPP = train_KPP.drop(['Сост.пед.сцепл.'],axis=1)
train_KPP = train_KPP.astype('float64')

In [13]:
X, y = train_KPP.drop(['target'], axis=1), train_KPP.target

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

In [15]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, stratify=y_test, test_size=0.3, random_state=42)

In [16]:
# model = CatBoostClassifier(
#     iterations=200, depth=10, learning_rate=0.1,
#    loss_function='MultiClass', verbose=100)
# model.fit(X_train, y_train,eval_set=(X_val,y_val),plot=True)

In [17]:
from joblib import load

model = load("model_kpp.pkl")

In [18]:
y_pred = model.predict(X_test)

In [19]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.8939038925156432
Classification Report:
               precision    recall  f1-score   support

         0.0       0.80      0.49      0.61     98995
         1.0       1.00      1.00      1.00     39838
         2.0       0.90      0.97      0.93    451839

    accuracy                           0.89    590672
   macro avg       0.90      0.82      0.85    590672
weighted avg       0.89      0.89      0.88    590672



In [20]:
y_pred_val = model.predict(X_val)

accuracy = accuracy_score(y_val, y_pred_val)
report = classification_report(y_val, y_pred_val)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.8942270468425335
Classification Report:
               precision    recall  f1-score   support

         0.0       0.80      0.49      0.61     42427
         1.0       1.00      1.00      1.00     17073
         2.0       0.90      0.97      0.93    193646

    accuracy                           0.89    253146
   macro avg       0.90      0.82      0.85    253146
weighted avg       0.89      0.89      0.88    253146



In [21]:
df_imp = pd.DataFrame({
    'name': X.columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

In [22]:
df_imp

,name,imp
1,КПП. Давление масла в системе смазки,37.865666
0,КПП. Температура масла,37.277350
4,"Давление в пневмостистеме (spn46), кПа",20.885262
2,Нейтраль КПП (spn3843),3.971722
3,Засоренность фильтра КПП (spn3847),0.000000
5,Недопустимый уровень масла в гидробаке (spn3850),0.000000
6,Аварийная температура масла в гидросистеме (sp...,0.000000
7,Аварийная температура масла ГТР (spn3867),0.000000


In [23]:
from joblib import dump

dump(model, "model_kpp.pkl")

['model_kpp.pkl']

In [24]:
y_proba_val = model.predict_proba(X_val)

In [25]:
df_fin = pd.DataFrame({
    'normal':y_proba_val[:, 0],
    'anomaly': y_proba_val[:, 1],
    'problem' : y_proba_val[:, 2]
})

In [26]:
df_fin['normal'] = df_fin['normal'].mean()
df_fin['anomaly'] = df_fin['anomaly'].mean()
df_fin['problem'] = df_fin['problem'].mean()

In [27]:
df_fin.drop_duplicates()

,normal,anomaly,problem
0,0.167978,0.067444,0.764577
